# ALL at once

Patch Generation + Noise Imprinted Image Saving

## For NOSE

In [19]:
import os
import cv2
import numpy as np
from pathlib import Path
from imutils import paths
from shutil import copyfile, copy
import dlib
from math import hypot

val_set = 'TestSet_cropped'

# Loading the Face Detector
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("/opt/hubshare/vectorly-share/shared/RealLife-SR/Adversarial-Attacks-on-Face-Recognition-Systems/InceptionResnet/shape_predictor_68_face_landmarks.dat")

bd_image = cv2.imread("/opt/hubshare/vectorly-share/shared/RealLife-SR/Adversarial-Attacks-on-Face-Recognition-Systems/InceptionResnet/NoisyPatch.png")
 
for img in paths.list_images(val_set):

    person_name = os.path.split(os.path.split(img)[0])[1]
    file_name = os.path.split(img)[1]

    #print(person_name, file_name)

    frame = cv2.imread(img)
    rows, cols, _ = frame.shape
    nose_mask = np.zeros((rows, cols), np.uint8)

    nose_mask.fill(0)
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = detector(frame)
    for face in faces:
        landmarks = predictor(gray_frame, face)

        # Nose coordinates
        top_nose = (landmarks.part(27).x, landmarks.part(27).y)
        center_nose = (landmarks.part(29).x, landmarks.part(29).y)

        left_eye_point = (landmarks.part(39).x, landmarks.part(39).y)
        right_eye_point = (landmarks.part(42).x, landmarks.part(42).y)

        nose_width = int(hypot(left_eye_point[0] - right_eye_point[0],
                                left_eye_point[1] - right_eye_point[1]))

        nose_height = int(nose_width * 1.73)   # 0.37 comes from H/W of the Image (373/100) - 1.73 is a random no

        # New nose position
        top_left = (int(center_nose[0] - nose_width / 2),
                        int(center_nose[1] - nose_height /2))

        bottom_right = (int(center_nose[0] + nose_width / 2),
                            int(center_nose[1] + nose_height / 2))

        # Adding the Band Aid Image
        bd_pic = cv2.resize(bd_image, (nose_width, nose_height))
        bd_pic_gray = cv2.cvtColor(bd_pic, cv2.COLOR_BGR2GRAY)
        _, nose_mask = cv2.threshold(bd_pic_gray, 25, 255, cv2.THRESH_BINARY_INV)

        nose_area = frame[top_left[1]: top_left[1] + nose_height,
                    top_left[0]: top_left[0] + nose_width]

        nose_area_no_nose = cv2.bitwise_and(nose_area, nose_area, mask=nose_mask)

        final_nose = cv2.add(nose_area_no_nose, bd_pic)
        #print("Final Nose Shape", final_nose.shape)

        frame[top_left[1]: top_left[1] + nose_height,
                    top_left[0]: top_left[0] + nose_width] = final_nose

        #cv2.imshow("Final Nose", final_nose)

    #print(frame.shape)
    #cv2.imshow("Frame", frame)

    #path = 'facenet_pytorch/data/PaperTesting/PatchonFace/Nose/30x18/{0}/{1}'.format(person_name, file_name)
    #print(path)
    cv2.imwrite('/opt/hubshare/vectorly-share/shared/RealLife-SR/Adversarial-Attacks-on-Face-Recognition-Systems/InceptionResnet/facenet_pytorch/data/PaperTesting/PatchonFace/Nose/30x18/{0}/{1}'.format(person_name, file_name), frame)
    #key = cv2.waitKey(0)

        #copy(img, os.path.join('facenet_pytorch/data/NoiseValidation/eps_1/',person_name ))

In [20]:
# val_set = 'TestSet_cropped/'

# from pathlib import Path

# for img in paths.list_images(val_set):
#     os.remove(img)